<a href="https://colab.research.google.com/github/427paul/ai_agent/blob/main/ai_agent_04_Memory.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install -U "langchain==0.3.*" "langchain-core==0.3.*" "langchain-community==0.3.*" "langgraph==0.3.*" "langchain-huggingface" "huggingface_hub" "sentence-transformers" wikipedia -q

  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 19.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 458.9/458.9 kB 25.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 50.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 148.2/148.2 kB 9.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 553.2/553.2 kB 29.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.8/45.8 kB 2.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.3/50.3 kB 3.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 66.5/66.5 kB 3.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.7/64.7 kB 5.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 51.0/51.0 kB 4.1 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
import os
def load_api_keys(filepath="api_key.txt"):
    with open(filepath, "r") as f:
        for line in f:
            line = line.strip()
            if line and "=" in line:
                key, value = line.split("=", 1)
                os.environ[key.strip()] = value.strip()

path = '/content/drive/MyDrive/LangGraph/'

# API 키 로드 및 환경변수 설정
load_api_keys(path + 'api_key.txt')

# example

In [4]:
from langchain.memory import ConversationBufferMemory

memory = ConversationBufferMemory()
memory.chat_memory.add_user_message("hi!")
memory.chat_memory.add_ai_message("what's up?")

print(memory.load_memory_variables({}))
"""
{'history': "Human: hi!\nAI: what's up?"}
"""

# memory key change
memory = ConversationBufferMemory(memory_key="chat_history")
memory.chat_memory.add_user_message("hi!")
memory.chat_memory.add_ai_message("what's up?")

print(memory.load_memory_variables({}))
"""
{'chat_history': "Human: hi!\nAI: what's up?"}
"""

# return in str type or list type
memory = ConversationBufferMemory(return_messages=True)
memory.chat_memory.add_user_message("hi!")
memory.chat_memory.add_ai_message("what's up?")

print(memory.load_memory_variables({}))
"""
{'history': [HumanMessage(content='hi!'), AIMessage(content="what's up?")]}
"""

{'history': "Human: hi!\nAI: what's up?"}
{'chat_history': "Human: hi!\nAI: what's up?"}
{'history': [HumanMessage(content='hi!', additional_kwargs={}, response_metadata={}), AIMessage(content="what's up?", additional_kwargs={}, response_metadata={})]}


/tmp/ipython-input-2122943195.py:3: LangChainDeprecationWarning: Please see the migration guide at: https://python.langchain.com/docs/versions/migrating_memory/
  memory = ConversationBufferMemory()


'\n{\'history\': [HumanMessage(content=\'hi!\'), AIMessage(content="what\'s up?")]}\n'

# adding memory

In [5]:
# Note as of 02/27/2024
# before you start you need to install the following
# pip install langchain==0.1.9 langchain-openai==0.0.8
from operator import itemgetter
from langchain_huggingface import HuggingFaceEndpoint, ChatHuggingFace
from langchain.memory import ConversationBufferMemory
from langchain.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain.schema.runnable import RunnableLambda, RunnablePassthrough

llm_ep = HuggingFaceEndpoint(repo_id="openai/gpt-oss-20b", task="text-generation")
model = ChatHuggingFace(llm=llm_ep)
prompt = ChatPromptTemplate.from_messages(
    [
        ("system", "You are a helpful chatbot"),
        MessagesPlaceholder(variable_name="history"),
        ("human", "{input}"),
    ]
)

memory = ConversationBufferMemory(return_messages=True)

chain = (
    RunnablePassthrough.assign(
        history=RunnableLambda(memory.load_memory_variables) | itemgetter("history")
    )
    | prompt
    | model
)

inputs = {"input": "Hello, My name is Joon"}
response = chain.invoke(inputs)
print(response)
"""
content="Hello Joon! It's nice to meet you. How can I assist you today?"
"""
memory.save_context(inputs, {"output": response.content}) # type: ignore

print(memory.load_memory_variables({}))
"""
{'history': [HumanMessage(content='Hello, My name is Joon'), AIMessage(content='Hello Joon! How can I assist you today?')]}
"""

inputs = {"input": "whats my name"}
response = chain.invoke(inputs)
print(response)
"""
content='Your name is Joon. How can I assist you further, Joon?
"""

content='Hello Joon! 👋 How can I assist you today?' additional_kwargs={} response_metadata={'token_usage': {'completion_tokens': 40, 'prompt_tokens': 86, 'total_tokens': 126}, 'model_name': 'openai/gpt-oss-20b', 'system_fingerprint': 'fp_334cc21c60', 'finish_reason': 'stop', 'logprobs': None} id='run--019c3183-a72c-7dc0-ab43-9a229a69d7d5-0' usage_metadata={'input_tokens': 86, 'output_tokens': 40, 'total_tokens': 126}
{'history': [HumanMessage(content='Hello, My name is Joon', additional_kwargs={}, response_metadata={}), AIMessage(content='Hello Joon! 👋 How can I assist you today?', additional_kwargs={}, response_metadata={})]}
content='You’re Joon. If you’d like me to remember that for later chats, just let me know!' additional_kwargs={} response_metadata={'token_usage': {'completion_tokens': 70, 'prompt_tokens': 113, 'total_tokens': 183}, 'model_name': 'openai/gpt-oss-20b', 'system_fingerprint': 'fp_3d50e63726', 'finish_reason': 'stop', 'logprobs': None} id='run--019c3183-a8d2-7000-a1

"\ncontent='Your name is Joon. How can I assist you further, Joon?\n"

# Conversation Buffer Window Memory

In [6]:
from langchain.memory import ConversationBufferWindowMemory

memory = ConversationBufferWindowMemory(k=1)
memory.save_context({"input": "hi"}, {"output": "whats up"})
memory.save_context({"input": "not much you"}, {"output": "not much"})

print(memory.load_memory_variables({}))
"""
{'history': 'Human: not much you\nAI: not much'}
"""

memory = ConversationBufferWindowMemory(k=1, return_messages=True)
memory.save_context({"input": "hi"}, {"output": "whats up"})
memory.save_context({"input": "not much you"}, {"output": "not much"})

print(memory.load_memory_variables({}))
"""
{'history': [HumanMessage(content='not much you'), AIMessage(content='not much')]}
"""


{'history': 'Human: not much you\nAI: not much'}
{'history': [HumanMessage(content='not much you', additional_kwargs={}, response_metadata={}), AIMessage(content='not much', additional_kwargs={}, response_metadata={})]}


/tmp/ipython-input-59787945.py:3: LangChainDeprecationWarning: Please see the migration guide at: https://python.langchain.com/docs/versions/migrating_memory/
  memory = ConversationBufferWindowMemory(k=1)


"\n{'history': [HumanMessage(content='not much you'), AIMessage(content='not much')]}\n"

# entity

In [ ]:
# Note as of 02/27/2024
# before you start you need to install the following
# pip install langchain==0.1.9 langchain-openai==0.0.8
from langchain_openai import OpenAI
from langchain.memory import ConversationEntityMemory

llm = OpenAI(temperature=0)
memory = ConversationEntityMemory(llm=llm)
_input = {"input": "Deven & Sam are working on a hackathon project"}
memory.load_memory_variables(_input)
memory.save_context(
    _input,
    {"output": " That sounds like a great project! What kind of project are they working on?"}
)

print(memory.load_memory_variables({"input": 'who is Sam'}))
"""
{'history': 'Human: Deven & Sam are working on a hackathon project\n
    AI:  That sounds like a great project! What kind of project are they working on?',
'entities': {'Sam': 'Sam is working on a hackathon project with Deven.'}}
"""

print(memory.load_memory_variables({"input": 'who is Deven'}))
"""
{'history': 'Human: Deven & Sam are working on a hackathon project\n
    AI:  That sounds like a great project! What kind of project are they working on?',
'entities': {
    'Deven': 'Deven is working on a hackathon project with Sam.',
    'Sam': 'Sam is working on a hackathon project with Deven.'}}
"""

In [8]:
# Note as of 02/27/2024
# before you start you need to install the following
# pip install langchain==0.1.9 langchain-openai==0.0.8
from langchain_huggingface import HuggingFaceEndpoint, ChatHuggingFace
from langchain.memory import ConversationEntityMemory

llm_ep = HuggingFaceEndpoint(repo_id="openai/gpt-oss-20b", task="text-generation")
llm = ChatHuggingFace(llm=llm_ep)
memory = ConversationEntityMemory(llm=llm)
_input = {"input": "Deven & Sam are working on a hackathon project"}
memory.load_memory_variables(_input)
memory.save_context(
    _input,
    {"output": " That sounds like a great project! What kind of project are they working on?"}
)

print(memory.load_memory_variables({"input": 'who is Sam'}))
"""
{'history': 'Human: Deven & Sam are working on a hackathon project\n
    AI:  That sounds like a great project! What kind of project are they working on?',
'entities': {'Sam': 'Sam is working on a hackathon project with Deven.'}}
"""

print(memory.load_memory_variables({"input": 'who is Deven'}))
"""
{'history': 'Human: Deven & Sam are working on a hackathon project\n
    AI:  That sounds like a great project! What kind of project are they working on?',
'entities': {
    'Deven': 'Deven is working on a hackathon project with Sam.',
    'Sam': 'Sam is working on a hackathon project with Deven.'}}
"""

/tmp/ipython-input-337027836.py:9: LangChainDeprecationWarning: Please see the migration guide at: https://python.langchain.com/docs/versions/migrating_memory/
  memory = ConversationEntityMemory(llm=llm)
/usr/local/lib/python3.12/dist-packages/pydantic/main.py:250: LangChainDeprecationWarning: Please see the migration guide at: https://python.langchain.com/docs/versions/migrating_memory/
  validated_self = self.__pydantic_validator__.validate_python(data, self_instance=self)


{'history': 'Human: Deven & Sam are working on a hackathon project\nAI:  That sounds like a great project! What kind of project are they working on?', 'entities': {'Sam': 'Sam is working on a hackathon project with Deven.'}}
{'history': 'Human: Deven & Sam are working on a hackathon project\nAI:  That sounds like a great project! What kind of project are they working on?', 'entities': {'Deven': 'Deven is collaborating with Sam on a hackathon project.'}}


"\n{'history': 'Human: Deven & Sam are working on a hackathon project\n\n    AI:  That sounds like a great project! What kind of project are they working on?', \n'entities': {\n    'Deven': 'Deven is working on a hackathon project with Sam.',\n    'Sam': 'Sam is working on a hackathon project with Deven.'}}\n"

# Connversation Summary Memory

In [ ]:
# Note as of 02/27/2024
# before you start you need to install the following
# pip install langchain==0.1.9 langchain-openai==0.0.8
from langchain.memory import ConversationSummaryMemory, ChatMessageHistory
from langchain_openai import OpenAI

memory = ConversationSummaryMemory(llm=OpenAI(temperature=0))
memory.save_context({"input": "hi"}, {"output": "whats up"})

print(memory.load_memory_variables({}))
"""
{'history': '\nThe human greets the AI, to which the AI responds.'}
"""

memory = ConversationSummaryMemory(llm=OpenAI(temperature=0), return_messages=True)
memory.save_context({"input": "hi"}, {"output": "whats up"})

# print(memory.load_memory_variables({}))
"""
{'history': [SystemMessage(content='\nThe human greets the AI, to which the AI responds.')]}
"""

# Directly use the memory
messages = memory.chat_memory.messages
"""
[HumanMessage(content='hi'), AIMessage(content='whats up')]
"""
previous_summary = ""
print(memory.predict_new_summary(messages, previous_summary))
"""
The human greets the AI, to which the AI responds.
"""

# Initializing with messages/existing summary
history = ChatMessageHistory()
history.add_user_message("hi")
history.add_ai_message("hi there!")

memory = ConversationSummaryMemory.from_messages(
    llm=OpenAI(temperature=0),
    chat_memory=history,
    return_messages=True
)
print(memory.buffer)
"""
The human greets the AI, to which the AI responds with a friendly greeting.
"""

# summary initialization
memory = ConversationSummaryMemory(
    llm=OpenAI(temperature=0),
    buffer="The human asks what the AI thinks of artificial intelligence. The AI thinks artificial intelligence is a force for good because it will help humans reach their full potential.",
    chat_memory=history,
    return_messages=True
)
print(memory.buffer)
"""
The human asks what the AI thinks of artificial intelligence. The AI thinks artificial intelligence is a force for good because it will help humans reach their full potential.
"""

In [11]:
# Note as of 02/27/2024
# before you start you need to install the following
# pip install langchain==0.1.9 langchain-openai==0.0.8
from langchain.memory import ConversationSummaryMemory, ChatMessageHistory
from langchain_huggingface import HuggingFaceEndpoint, ChatHuggingFace

llm_ep = HuggingFaceEndpoint(repo_id="openai/gpt-oss-20b", task="text-generation")
model = ChatHuggingFace(llm=llm_ep)

memory = ConversationSummaryMemory(llm=model)
memory.save_context({"input": "hi"}, {"output": "whats up"})

print(memory.load_memory_variables({}))
"""
{'history': '\nThe human greets the AI, to which the AI responds.'}
"""

memory = ConversationSummaryMemory(llm=model, return_messages=True)
memory.save_context({"input": "hi"}, {"output": "whats up"})

# print(memory.load_memory_variables({}))
"""
{'history': [SystemMessage(content='\nThe human greets the AI, to which the AI responds.')]}
"""

# Directly use the memory
messages = memory.chat_memory.messages
"""
[HumanMessage(content='hi'), AIMessage(content='whats up')]
"""
previous_summary = ""
print(memory.predict_new_summary(messages, previous_summary))
"""
The human greets the AI, to which the AI responds.
"""

# Initializing with messages/existing summary
history = ChatMessageHistory()
history.add_user_message("hi")
history.add_ai_message("hi there!")

memory = ConversationSummaryMemory.from_messages(
    llm=model,
    chat_memory=history,
    return_messages=True
)
print(memory.buffer)
"""
The human greets the AI, to which the AI responds with a friendly greeting.
"""

# summary initialization
memory = ConversationSummaryMemory(
    llm=model,
    buffer="The human asks what the AI thinks of artificial intelligence. The AI thinks artificial intelligence is a force for good because it will help humans reach their full potential.",
    chat_memory=history,
    return_messages=True
)
print(memory.buffer)
"""
The human asks what the AI thinks of artificial intelligence. The AI thinks artificial intelligence is a force for good because it will help humans reach their full potential.
"""

/tmp/ipython-input-3345462247.py:10: LangChainDeprecationWarning: Please see the migration guide at: https://python.langchain.com/docs/versions/migrating_memory/
  memory = ConversationSummaryMemory(llm=model)


{'history': 'The human says “hi.” The AI replies, “whats up.”'}
The human greets the AI, and the AI responds with a casual “whats up.”
New summary:  
The human says “hi,” and the AI replies “hi there!”
The human asks what the AI thinks of artificial intelligence. The AI thinks artificial intelligence is a force for good because it will help humans reach their full potential.


'\nThe human asks what the AI thinks of artificial intelligence. The AI thinks artificial intelligence is a force for good because it will help humans reach their full potential.\n'

# Conversation Summary Buffer Memory

In [ ]:
# Note as of 02/27/2024
# before you start you need to install the following
# pip install langchain==0.1.9 langchain-openai==0.0.8
from langchain_openai import OpenAI
from langchain.memory import ConversationSummaryBufferMemory

llm = OpenAI()

memory = ConversationSummaryBufferMemory(llm=llm, max_token_limit=10)
memory.save_context({"input": "hi"}, {"output": "whats up"})
memory.save_context({"input": "not much you"}, {"output": "not much"})

print(memory.load_memory_variables({}))
"""
{'history': "System: \nThe human greets the AI and the AI responds asking what's up.\nHuman: not much you\nAI: not much"}
"""

memory = ConversationSummaryBufferMemory(
    llm=llm, max_token_limit=10, return_messages=True
)
memory.save_context({"input": "hi"}, {"output": "whats up"})
memory.save_context({"input": "not much you"}, {"output": "not much"})

print(memory.load_memory_variables({}))
"""
{'history': [
    SystemMessage(content="\nThe human greets the AI. The AI responds by asking what's up."),
    HumanMessage(content='not much you'),
    AIMessage(content='not much')]}
"""

messages = memory.chat_memory.messages
previous_summary = ""
print(memory.predict_new_summary(messages, previous_summary))
"""
The human and AI exchange pleasantries and express that there is not much going on.
"""

In [12]:
# Note as of 02/27/2024
# before you start you need to install the following
# pip install langchain==0.1.9 langchain-openai==0.0.8
from langchain_huggingface import HuggingFaceEndpoint, ChatHuggingFace
from langchain.memory import ConversationSummaryBufferMemory

llm_ep = HuggingFaceEndpoint(repo_id="openai/gpt-oss-20b", task="text-generation")
model = ChatHuggingFace(llm=llm_ep)

memory = ConversationSummaryBufferMemory(llm=llm, max_token_limit=10)
memory.save_context({"input": "hi"}, {"output": "whats up"})
memory.save_context({"input": "not much you"}, {"output": "not much"})

print(memory.load_memory_variables({}))
"""
{'history': "System: \nThe human greets the AI and the AI responds asking what's up.\nHuman: not much you\nAI: not much"}
"""

memory = ConversationSummaryBufferMemory(
    llm=llm, max_token_limit=10, return_messages=True
)
memory.save_context({"input": "hi"}, {"output": "whats up"})
memory.save_context({"input": "not much you"}, {"output": "not much"})

print(memory.load_memory_variables({}))
"""
{'history': [
    SystemMessage(content="\nThe human greets the AI. The AI responds by asking what's up."),
    HumanMessage(content='not much you'),
    AIMessage(content='not much')]}
"""

messages = memory.chat_memory.messages
previous_summary = ""
print(memory.predict_new_summary(messages, previous_summary))
"""
The human and AI exchange pleasantries and express that there is not much going on.
"""

/tmp/ipython-input-3799790583.py:10: LangChainDeprecationWarning: Please see the migration guide at: https://python.langchain.com/docs/versions/migrating_memory/
  memory = ConversationSummaryBufferMemory(llm=llm, max_token_limit=10)
/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

{'history': 'System: Human says hi; AI asks “what’s up.”\nHuman: not much you\nAI: not much'}
{'history': [SystemMessage(content='The human says hi, and the AI responds with “whats up.”', additional_kwargs={}, response_metadata={}), HumanMessage(content='not much you', additional_kwargs={}, response_metadata={}), AIMessage(content='not much', additional_kwargs={}, response_metadata={})]}
New summary:
The human says they have nothing much to say, and the AI responds with a brief “not much.”


'\nThe human and AI exchange pleasantries and express that there is not much going on.\n'